# Lab 03_2: Qubit-Resonator System (Planar Chip)

### developed by Seong Hyeon Park (pajoheji0909@snu.ac.kr)

### Please find the attached homework at the end of this tutorial code

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from qiskit_metal import designs, MetalGUI, Dict

design = designs.DesignPlanar()
design.overwrite_enabled = True

In [ ]:
# set the overall layout size (X)
design.chips.main.size.size_x = '6mm'
# set the overall layout size (Y)
design.chips.main.size.size_y = '6mm'
# set the substrate thickness (Z)
design.chips.main.size.size_z = '-400um'

## start GUI of Qiskit-Metal and ANSYS HFSS

In [ ]:
# start GUI
gui = MetalGUI(design)

In [ ]:
hfss = design.renderers.hfss
# start ANSYS HFSS
hfss.start()

In [ ]:
import pyEPR as epr
import qiskit_metal as metal

# design your qubit and resonator for readout!

In [ ]:
# for readout resonator (CPW)
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround

# there are various in-built qubit types in Qiskit-Metal, you should try it by yourself
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL

In [ ]:
design.delete_all_components()

# you can directly write down the options
# qubit
q1 = TransmonPocketCL(design, 'Q1', 
                      options = dict(pad_width = '425 um', pocket_height = '650um',
                                     connection_pads=dict(readout = dict(loc_W=+1,loc_H=+1, pad_width='200um'))
                                     )
                      )

# open to ground for quarter-wavelength CPW resonator
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))

# CPW resonator
readout = RouteMeander(design, 'readout',  Dict(total_length='6 mm',
                                                hfss_wire_bonds = True,
                                                fillet='90 um',
                                                lead = dict(start_straight='100um'),
                                                pin_inputs=Dict(start_pin=Dict(component='Q1', pin='readout'), end_pin=Dict(component='open_to_ground', pin='open'))
                                                )
                       )

gui.rebuild()
gui.autoscale()

# take a screenshot of the GUI
gui.screenshot()

# Realistic Single-Qubit Device Design with Qubit Control Line and Two-Port Readout Transmission Line

### A device should have signal input/output pads for qubit readout as well as qubit control
### Refer to below papers for further details:

* [two-port readout with individual qubit control line] L DiCarlo *et al.,* "Demonstration of two-qubit algorithms with a superconducting quantum processor," *Nature,* **460**, 240-244 (2009) DOI: https://doi.org/10.1038/nature08121
* [single-port readout with individual qubit control line] Y Sunada *et al.,* "Photon-Noise-Tolerant Dispersive Readout of a Superconducting Qubit Using a Nonlinear Purcell Filter," *PRX Quantum,* **5**, 010307 (2024) DOI: https://doi.org/10.1103/PRXQuantum.5.010307

In case of single-port readout protocol, you should have `microwave circulators` to separate the input readout pulse from the reflected readout pulse

In [ ]:
# additional package loads for readout transmission line design
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.terminations.launchpad_wb_driven import LaunchpadWirebondDriven

# built-in transmon qubit design
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6

In [ ]:
# transmon qubit's Josephson junction parameters
tr_Ljj = '13 nH' ; tr_Cjj = '2 fF'
# CPW characteristic parameters
cpw_w = '10 um'; cpw_g = '6 um'
# readout resonator parameters
res1_l = '3.5 mm'; res1_coup_l = '600 um'; res1_coup_g = '5 um'

design.delete_all_components()

# options for realistic signal input/output port (NOTE: width of the launchpad should be larger than 300 um in practice)
launch_options = dict(
    chip='main', 
    pos_x="-2.12 mm", 
    pos_y="0mm",  
    orientation='360', 
    lead_length='0um',
    pad_width='300um',
    pad_height='300um',
    pad_gap='180um',
    taper_height='300um'
)
RP1 = LaunchpadWirebondDriven(design, 'RP1', options = launch_options)

# options for realistic signal input/output port (NOTE: width of the launchpad should be larger than 300 um in practice)
launch_options = dict(
    chip='main', 
    pos_x="2.12 mm", 
    pos_y="0mm",  
    orientation='180', 
    lead_length='30um',
    pad_width='300um',
    pad_height='300um',
    pad_gap='180um',
    taper_height='300um'
)
RP2 = LaunchpadWirebondDriven(design, 'RP2', options = launch_options)

# options for built-in capacitive coupling geometry
# you can draw the capacitively coupled geometry using the open/short_to_ground components (see lab01_3)
clt_options = dict(
    chip='main', 
    pos_x="0mm", 
    pos_y="0mm",
    prime_width=cpw_w,
    prime_gap=cpw_g,
    second_width=cpw_w,
    second_gap=cpw_g,
    coupling_space=res1_coup_g,
    coupling_length=res1_coup_l,
    down_length='100um',
    fillet='40um',
    mirror=False,
    open_termination=False,
    hfss_wire_bonds = False
)
CLT1 = CoupledLineTee(design, 'CLT1', options = clt_options)

# draw the readout transmission line (part 1)
route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='RP1', pin='tie'),
                                     end_pin=Dict(component='CLT1',pin='prime_start')),
                     trace_width=cpw_w, trace_gap=cpw_g)
TL1 = RouteStraight(design, 'TL1', options=route_options)

# draw the readout transmission line (part 2)
route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='CLT1', pin='prime_end'),
                                     end_pin=Dict(component='RP2',pin='tie')),
                     trace_width=cpw_w, trace_gap=cpw_g)
TL2 = RouteStraight(design, 'TL2', options=route_options)

# draw transmon qubit
tr_options = dict(pos_x="0mm", pos_y="-1.5 mm", orientation = 180,
                  pad_width = '600um', pad_height = '60um', pad_gap = '30um',
                  pocket_height = '300um', pocket_width = '700um', inductor_width = '10um',
                  connection_pads=dict(readout = dict(loc_W=0, loc_H=-1, 
                                                      pad_height='20um', pad_width='200um', pad_gap='20um', pad_shift='0um')),
                  hfss_inductance = tr_Ljj,
                  hfss_capacitance = tr_Cjj
                 )
Q1 = TransmonPocket6(design, 'Q1', options = tr_options)

# draw quarter-wavelength CPW resonator for dispersive qubit readout
route_options = Dict(chip='main',
                     hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='Q1',pin='readout'),
                                     end_pin=Dict(component='CLT1',pin='second_end')),
                     trace_width=cpw_w,
                     trace_gap=cpw_g,
                     total_length=res1_l,
                     fillet="80um",
                     meander = dict(spacing = '200um', asymmetry = '0um'),
                     lead = dict(start_straight = '100um', end_straight = '50 um'))
RR1 = RouteMeander(design, 'RR1', options=route_options)

# options for realistic qubit control line port
launch_options = dict(chip='main', pos_x="0mm", pos_y="-2.38mm", orientation='90', 
                      lead_length='0um',
                      pad_width='200um', pad_height='200um', pad_gap='120um', taper_height='200um'
                      )
QCTRLP1 = LaunchpadWirebondDriven(design, 'QCTRLP1', options = launch_options)

OTG = OpenToGround(design, name='OTG1', options=dict(pos_x='0mm',  pos_y='-1.7 mm', orientation='90'))

route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='OTG1', pin='open'),
                                     end_pin=Dict(component='QCTRLP1',pin='tie')),
                     trace_width=cpw_w, trace_gap=cpw_g)
QCTRLR1 = RouteStraight(design, 'QCTRLR1', options=route_options)

gui.rebuild()
gui.autoscale()

# method (1): Energy Participation Ratio (EPR)

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis

# project name
proj_name = "lab03_transmon_readout"

eig_s = EPRanalysis(design, "hfss")
hfss = eig_s.sim.renderer

try:
    hfss.start()
except:
    hfss.activate_design(proj_name)

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis

# define the EPR analysis class
eig_s = EPRanalysis(design, "hfss")

# define the renderer program for the eigenmode simulation
hfss = eig_s.sim.renderer

# try-except to avoid common(?) error during the HFSS rendering
try:
    hfss.start()
except:
    hfss.activate_design(proj_name)

# NOTE: you should define the junction in HFSS
eig_s.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_s.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

# define the setup
em_s = eig_s.sim.setup
em_s.name = "setup_custom"

# minimum searching frequency
em_s.min_freq_ghz = 3

# target eigenmode number
# NOTE: now you have two components... you should find 'two' eigenmodes
em_s.n_modes = 2

# maximum pass number
em_s.max_passes = 15

# criteria for eigenmode simulation convergence: maximum delta F_eigen between passes < value
em_s.max_delta_f = 0.5

# simulation minimum converged passes 
em_s.min_converged = 2

# simulation order 0: point, 1: line, 2: face
em_s.basis_order = 1

# Design variables can also be added in for direct simulation sweeps. Here, we have to set the junction inductance and capacitance
em_s.vars = Dict({'Lj': tr_Ljj, 'Cj': tr_Cjj})


In [ ]:
eig_s.sim._render(name=proj_name,                         # design name
                  selection = [],                         # if None --> all
                  solution_type='eigenmode',              # NOTE: case sensitive
                  vars_to_initialize=em_s.vars,           # set variables
                  open_pins=[],                           # set open pins
                  port_list=[],                           # set ports, for eigenmode simulations --> R = 50 Ohm boundary conditions 
                  box_plus_buffer = False)                # if False --> set the overall layout as you defined above. If True --> set marginal chip sizes as you set here.

In [ ]:
# mesh setting name, components, maximum mesh length
hfss.modeler.mesh_length('qubitpad_mesh_setting', 
                         ['pad_bot_Q1', 'pad_top_Q1', 'readout_connector_pad_Q1'], 
                         MaxLength='0.1 mm')

hfss.modeler.mesh_length('cpw_mesh_setting', 
                         ['trace_RR1', 'prime_cpw_CLT1', 'second_cpw_CLT1'], 
                         MaxLength='0.05 mm')

In [ ]:
eig_s.sim._analyze()

In [ ]:
pinfo = hfss.pinfo
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj', 'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied

In [ ]:
eprd = epr.DistributedAnalysis(pinfo)
eprd.do_EPR_analysis()

In [ ]:
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 10)

# Method (2): Capacitance Analysis and LOM derivation using the analysis package

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [ ]:
# example: update single setting
c1.sim.setup.max_passes = 20
c1.sim.setup.freq_ghz = 5

# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

In [ ]:
# you should define the open terminations, where coupling structures of a qubit end
c1.sim.run(components=['Q1'], open_terminations=[('Q1', 'readout')])

# c1.setup.run <- direct access
c1.sim.print_run_args()

In [ ]:
c1.sim.capacitance_matrix

In [ ]:
# NOTE: you should define the same values for Josephson junction parameters!
c1.setup.junctions = Dict({'Lj': 13, 'Cj': 2}) # Lj unit: nH and Cj unit: fF

# NOTE: you have to define the readout resonator's frequency!
c1.setup.freq_readout = 6.33 # unit: GHz
c1.setup.freq_bus = []      # if there is additional coupling bus resonators, you should define their frequencies

c1.run_lom()
c1.lumped_oscillator_all

# Method (3): New LOM analysis

In [ ]:
import scqubits as scq

from scipy.constants import speed_of_light as c_light
from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem, QuantumSystemRegistry

In [ ]:
QuantumSystemRegistry.registry()

### 1. load transmon cell Q3d simulation results and set cell objects corresponding to the capacitance simulation results 
Loading the Maxwell capacitance matrices for the design from ANSYS Q3D

For a simple introduction on Maxwell capacitance matrix, check out the following resources:
https://www.fastfieldsolvers.com/Papers/The_Maxwell_Capacitance_Matrix_WP110301_R02.pdf

The following three parameters, `ind_dict`, `jj_dict`, `cj_dict`, all have the same structure. Each is a dictionary where the keys are tuples, giving the nodes that a junction is in between, and the values specifying the relevant values associated with the junction. `ind_dict` lets you specify the junction inductance in nH; `jj_dict` specifies the Josephson junction name (you can give the junction any name you wish; just need to be consistent with the name); `cj_dict` specifies the junction capacitance in fF.

In [ ]:
# cell 1: transmon qubit Q1

opt1 = dict(node_rename = {'readout_connector_pad_Q1': 'coupler_RR1'}, 
            cap_mat = c1.sim.capacitance_matrix,
            ind_dict = {('pad_top_Q1', 'pad_bot_Q1'):13},  # junction inductance in nH
            jj_dict = {('pad_top_Q1', 'pad_bot_Q1'):'j1'},
            cj_dict = {('pad_top_Q1', 'pad_bot_Q1'):2},    # junction capacitance in fF
            )

cell_1 = Cell(opt1)

### 2. Create subsystems
#### Creating the four subsystems, corresponding to the qubit-readout resonator

`Subsystem` takes three required arguments. The four currently supported system types are `TRANSMON`, `FLUXONIUM`, `TL_RESONATOR` (transmission line resonator) and `LUMPED_RESONATOR`. `nodes` lets you specify which node the subsystem should be mapped to in the cells. They should be consistent with the node names you have given previously. `q_opts` lets specify any optional parameters you want to give. For example, for qubits, you can provide `scqubits` parameters such as `ncut`, `truncated_dim` here.

In [ ]:
# subsystem 1: transmon qubit Q1
transmon_Q1 = Subsystem(name='Q1', sys_type='TRANSMON', nodes=['j1'])

# subsystem 2: readout resonator RR1
q_opts = dict(f_res = 6.33,           # resonator dressed frequency in GHz
              Z0 = 50,                # characteristic impedance in Ohm
              vp = 0.404314 * c_light # phase velocity 
)
res_RR1 = Subsystem(name='RR1', sys_type='TL_RESONATOR', nodes=['coupler_RR1'], q_opts=q_opts)

### 3. Create the composite system from the cells and the subsystems

The LOM analysis will automatically remove all non-dynamic nodes. These are nodes that are either exclusively connected to only capacitors or only inductors and are not true degrees of freedom. 

please check out https://arxiv.org/pdf/2103.10344.pdf or https://cpb-us-w2.wpmucdn.com/campuspress.yale.edu/dist/2/3627/files/2020/10/Vool_Intro_quantum_electromagnetic_circuits.pdf for more information on this

Since we didn't (and didn't have to) simulate the readout resonator, the node, `coupler_RR1`, connected only to other nodes capacitively as specified by the Maxwell capacitance matrices, would be eliminated. But they are actually dynamic nodes, connected to the inductors (not simulated) of the respective transmission lines and correspond to subsystems that we want to include in the Hamiltonian of the composite system, hence we list them as nodes to force keep with the parameter `nodes_force_keep`.

In [ ]:
composite_sys = CompositeSystem(subsystems=[transmon_Q1, res_RR1], 
                                cells=[cell_1], 
                                grd_node='ground_main_plane',
                                nodes_force_keep=['coupler_RR1']
                                )

The `circuitGraph` object encapsulates the lumped model circuit analysis (i.e., LOM analysis) and contain the intermediate as well as final L and C matrices, their inverses needed to construct the Hamiltonian of the composite system. For more details on the meaning and calculation of these matrices, check out https://arxiv.org/pdf/2103.10344.pdf.

Just to note that you can use the analysis without needing to know any detail about this object.

In [ ]:
cg = composite_sys.circuitGraph()
print(cg)

### 4. Generate the hilberspace from the composite system, leveraging the scqubits package

In [ ]:
hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

`add_interaction()` adds the interaction terms between the subsystems. Currently, capacitive coupling is supported (which is extracted by from off-diagonal elements in the C matrices, see *eqn 12, 13* in https://arxiv.org/pdf/2103.10344.pdf ) and contribute to the interaction.

In [ ]:
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

### 5. Print the results

Print the calculated Hamiltonian parameters from diagonalized composite system Hamiltonian.

The diagonal elements of the $\chi$ matrix are the anharmonicities of the respective subsystems and the off-diagonal the dispersive shifts between them. 

In [ ]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)

The $\chi$'s between the subsystems are based on the coupling strengths, $\it{g}$'s between them (which are computed using the coupling capacitance (currently capacitive coupling is supported) and zero point fluctuations of the subsystem's charge operator at the coupling location).

In [ ]:
composite_sys.compute_gs()

# LAB HOMEWORK

### Transmon qubit-resonator system simulations. You should achieve the target parameters within 5\%. The overall circuit layout, electric field distributions, simulations settings in ANSYS (including $L_J, C_J$ values), and the convergence plot vs. adaptive pass should be attached to the report.

1. Design a double pad qubit and a readout resonator to have $\omega_q/2\pi=4.5$ GHz, $\omega_r/2\pi=6.5$ GHz, $\chi/2\pi=5$ MHz from the `EPR` method. You have to modify the geometric features of a transmon qubit and a readout resonator.
1. Design a Xmon qubit (`from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross`) and a readout resonator to have $\omega_q/2\pi=4.5$ GHz, $\omega_r/2\pi=6.5$ GHz, $\chi/2\pi=5$ MHz from the `EPR` method. You have to modify the geometric features of a transmon qubit and a readout resonator (also Josephson junction parameters, if necessary).
2. Modify the substrate material into `sapphire` with a dielectric constant of `10`. Then, design a double pad qubit and a readout resonator to have $\omega_q/2\pi=4.5$ GHz, $\omega_r/2\pi=6.0$ GHz, $\chi/2\pi=2$ MHz from the `EPR` method. You have to modify the geometric features of a transmon qubit and a readout resonator.
3. Design a double pad qubit and a readout resonator to have $\omega_q/2\pi=5.0$ GHz, while varying the readout resonator frequency $\omega_r$ to obtain $\chi/2\pi=5$ MHz from the `LOM` and `NEW LOM` methods. You have to modify the geometric features of transmon qubit and Josephson junction parameters.